# [1.1] Get the list of movies

In [3]:
from bs4 import BeautifulSoup
import requests
import json

In [2]:
soup = BeautifulSoup(open('C:/Users/leona/Desktop/ADMHMK-3/movies2.html'), "html.parser")

In [3]:
soup.head()

[<style>
 table, th, td {border: 1px solid black;}</style>,
 <link href="https://www.w3schools.com/browserref.css" rel="stylesheet" type="text/css"/>]

In [4]:
lst_a = soup.select('a')

In [5]:
lst_a

[<a href="https://en.wikipedia.org/wiki/10_to_Midnight">https://en.wikipedia.org/wiki/10_to_Midnight</a>,
 <a href="https://en.wikipedia.org/wiki/All_the_Right_Moves_(film)">https://en.wikipedia.org/wiki/All_the_Right_Moves_(film)</a>,
 <a href="https://en.wikipedia.org/wiki/Americana_(film)">https://en.wikipedia.org/wiki/Americana_(film)</a>,
 <a href="https://en.wikipedia.org/wiki/Amityville_3-D">https://en.wikipedia.org/wiki/Amityville_3-D</a>,
 <a href="https://en.wikipedia.org/wiki/Anna_to_the_Infinite_Power">https://en.wikipedia.org/wiki/Anna_to_the_Infinite_Power</a>,
 <a href="https://en.wikipedia.org/wiki/Baby_It%27s_You_(film)">https://en.wikipedia.org/wiki/Baby_It%27s_You_(film)</a>,
 <a href="https://en.wikipedia.org/wiki/Bad_Boys_(1983_film)">https://en.wikipedia.org/wiki/Bad_Boys_(1983_film)</a>,
 <a href="https://en.wikipedia.org/wiki/Better_Late_Than_Never_(film)">https://en.wikipedia.org/wiki/Better_Late_Than_Never_(film)</a>,
 <a href="https://en.wikipedia.org/wiki/Th

In [6]:
urls = []
for i in lst_a:
    urls.append(i.get('href'))

In [7]:
#urls#Now I have a list with all the urls, so let's start the next point!

# [1.2] Crawl Wikipedia

In [9]:
from urllib.error import URLError, HTTPError, ContentTooShortError
import time
k=0
for i in range(len(urls)):
    try:
        ur_l = requests.get(urls[i])
        soup = BeautifulSoup(ur_l.content, 'html.parser')
        soup = soup.prettify("utf-8")        
        stringa = 'Articles/article-'+str(k)+'.html'
        k = k+1
        Html_file= open(stringa, "wb")
        Html_file.write(soup)
        Html_file.close()
    except(URLError,HTTPError, ContentTooShortError)  as e:
        html = None
    time.sleep(0.001)   

In [10]:
len(urls)

10000

# [1.3] Parse downloaded pages

In [35]:
myfile = open("Articles/article-0.html")
#soup = BeautifulSoup(myfile, 'html.parser')

In [15]:
import csv
import io
for i in range(6429, 10000):
    filename = "Articles/article-"+str(i)+".html"
    with open(filename, encoding="utf-8") as f:
        data = f.read()
        soup = BeautifulSoup(data, 'html.parser')
        #Now That I opened the file I have to look for the section asked
        titlepage = soup.title.string
        titleonly = titlepage.split("- Wikipedia")
        titlepage = titleonly[0].strip()
        intro = ''
        plot = ''
        start = soup.find('p')
        intro = start
        intro1 = start.text
        #print('EILa')
        B = ''
        B = intro.find_next_sibling('h2')
        if(B!=None and B.find_next_sibling('p')):
            C = B.find_next_sibling('p')
            #print(intro1)
            #print(intro.find_next_sibling('p'))
            #print(B)
            while(C != intro.find_next_sibling('p')): 
                intro1 = intro1 + intro.find_next_sibling('p').text
                intro = intro.find_next_sibling('p')
            plot = ''            
            plot = B
        #print(intro)
            plot1 = ''
            compare = ''
            if(B.find_next_sibling('h2')):
                compare = B.find_next_sibling('h2')
                compareto = compare.find_next_sibling('p')
                #plot1 = B.find_next_sibling('p').text
                #print(compareto)
                #print(plot1)
                while(compareto != plot.find_next_sibling('p')):
                    plot1 = plot1 + plot.find_next_sibling('p').text
                    #print(plot1)
                    plot = plot.find_next_sibling('p')
        if(intro1 == ''):
            intro1 = "NA"
        if plot1 == '':
            plot1 = "NA"
        intro = intro1
        plot = plot1
        film_name = 'NA'
        director = "NA"
        producer = "NA"
        writer = "NA"
        starring = "NA"
        music = "NA"
        release_date = "NA"
        runtime = "NA"
        country = "NA"
        language = "NA"
        budget = "NA"
#'film_name', 'director', 'producer','writer', 'starring', 'music', 'release date', 'runtime', 'country', 'language', 'budget'
        for link in soup.find_all('tr'):
            if soup.find('th',{'class': ['summary']})!= None:
                    film_name = soup.find('th',{'class': ['summary']} ).text.strip()
            if link.th:
                if(link.th.text.strip() == 'Directed by'):
                     director = link.td.text.strip()
                elif(link.th.text.strip() == 'Produced by'):
                      producer = link.td.text.strip()
                elif(link.th.text.strip() == 'Written by'):
                    writer = link.td.text.strip()
                elif(link.th.text.strip() == 'Starring'):
                    starring = link.td.text.strip()
                elif(link.th.text.strip() == 'Music by'):
                     music = link.td.text.strip()               
                elif(link.th.text.strip() == 'Release date'):
                    if link.td:
                        release_date = link.td.text.strip()
                elif(link.th.text.strip() == 'Running time'):
                    runtime = link.td.text.strip()
                elif(link.th.text.strip() == 'Country'):
                    if link.td:
                        country = link.td.text.strip()
                elif(link.th.text.strip() == 'Language'):
                    if link.td:
                        language = link.td.text.strip()
                elif(link.th.text.strip() == 'Budget'):
                    budget = link.td.text.strip()
        
        tsvname = 'Tsvfiles/'+'film'+str(i)+'.tsv'
        with io.open(tsvname, "w", encoding="utf-8") as out_file:
        #with open(tsvname, 'w') as out_file:
            tsv_writer = csv.writer(out_file, delimiter='\t')
            tsv_writer.writerow(['title','intro', 'plot', 'film_name', 'director', 'producer','writer', 'starring', 'music', 'release date', 'runtime', 'country', 'language', 'budget'])
            tsv_writer.writerow([titlepage, intro, plot, film_name, director, producer, writer, starring, music, release_date, runtime, 
                 country, language, budget])


In [16]:
titlepage

'What Became of Jack and Jill?'

In [17]:
intro

'\n\n\n\n         What Became of Jack and Jill?\n        \n\n       is a 1972 British\n       \n        horror film\n       \n       directed by\n       \n        Bill Bain\n       \n       and starring\n       \n        Mona Washbourne\n       \n       ,\n       \n        Paul Nicholas\n       \n       , and\n       \n        Vanessa Howard\n       \n       .\n       \n\n         [1]\n        \n\n       It was part of an abandoned attempt by\n       \n        Amicus Pictures\n       \n       to compete with\n       \n        Hammer Studios\n       \n       by breaking into the\n       \n        grindhouse\n       \n       market. Studio executives were ultimately too disturbed by the final product to release it under the Amicus name, and they sold the film to\n       \n        20th Century Fox\n       \n       .\n      '

this is just to try stuff.

In [18]:
intro = ''
plot = ''
start = soup.find('p')
intro = start
intro1 = start.text
#print('EILa')
B = ''
B = intro.find_next_sibling('h2')
C = B.find_next_sibling('p')
#print(intro1)
#print(intro.find_next_sibling('p'))
#print(B)
while(C != intro.find_next_sibling('p')): 
            intro1= intro1 + intro.find_next_sibling('p').text
            intro = intro.find_next_sibling('p')
plot = ''            
plot = B
#print(intro)
plot1 = ''
compare = ''
compare = B.find_next_sibling('h2')
compareto = compare.find_next_sibling('p')
#plot1 = B.find_next_sibling('p').text
#print(compareto)
#print(plot1)
while(compareto != plot.find_next_sibling('p')):
        plot1 = plot1 + plot.find_next_sibling('p').text
        #print(plot1)
        plot = plot.find_next_sibling('p')
        #print(plot)
print(intro1)
print("POI PLOT")
print(plot1)





         What Became of Jack and Jill?
        

       is a 1972 British
       
        horror film
       
       directed by
       
        Bill Bain
       
       and starring
       
        Mona Washbourne
       
       ,
       
        Paul Nicholas
       
       , and
       
        Vanessa Howard
       
       .
       

         [1]
        

       It was part of an abandoned attempt by
       
        Amicus Pictures
       
       to compete with
       
        Hammer Studios
       
       by breaking into the
       
        grindhouse
       
       market. Studio executives were ultimately too disturbed by the final product to release it under the Amicus name, and they sold the film to
       
        20th Century Fox
       
       .
      
POI PLOT

       Johnnie Tallent is a callous young
       
        mod
       
       who lives with his elderly, invalid grandmother, Alice. Lazy and unmotivated, Johnnie dedicates most of his time to taking care of 

In [13]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [52]:
import pandas as pd
dataset1 = pd.read_csv('Cleantsv/filmclean-8.tsv', delimiter='\t')
#dataset1 = pd.read_csv('Tsvfiles/film1', delimiter='\t')
dataset1

,title,intro,plot,film_name,director,producer,writer,starring,music,release date,runtime,country,language,budget
0,big chill film,big chill 1983 american comedi drama film dire...,harold cooper bath young son wife dr sarah coo...,big chill,lawrenc kasdan,michael shamberg,lawrenc kasdan barbara benedek,tom bereng glenn close jeff goldblum william h...,bill conti,septemb 28 1983 1983 09 28,105 minut,unit state,english,8 million 1


In [35]:
words[13]

'budget 10 midnight'

In [50]:
import io 
import nltk
import csv
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
#word_tokenize accepts a string as an input, not a file. 
stop_words = set(stopwords.words('english')) 
from nltk.stem import PorterStemmer 
#from nltk.tokenize import word_tokenize 
 
ps = PorterStemmer() 
def preprocess(sentence):
    #ps = PorterStemmer()
    sentence = sentence.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(sentence)
    filtered_words = [ps.stem(w) for w in tokens if not w in stopwords.words('english')]
    return " ".join(filtered_words)
for i in range(10000):
    file1 = open('Tsvfiles/film'+str(i)+'.tsv', encoding="utf8") 
    line = file1.read()# Use this to read file content as a stream: 
    #print(line)
    words = line.split('\t') 
    #print(words)
    for j in range(13, len(words)):
        words[j] = preprocess(words[j])
        if j ==13:
            words[j] = words[j].replace('budget ','')
           # words[j]=words[j][1]
        #print(words[j])
        #word = word.split()
        #for r in word:
           # if r in stop_words:
                #word.remove(r)
                #print(r)
            
    #print(words)    
    tsvname = "Cleantsv/filmclean-"+str(i)+'.tsv'
    with io.open(tsvname, "w", encoding="utf-8") as out_file:
        #with open(tsvname, 'w') as out_file:
            tsv_writer = csv.writer(out_file, delimiter='\t')
            tsv_writer.writerow(['title','intro', 'plot', 'film_name', 'director', 'producer','writer', 'starring', 'music', 'release date', 'runtime', 'country', 'language', 'budget'])
            tsv_writer.writerow(words[13:])

In [53]:
words[14]

'becam jack jill 1972 british horror film direct bill bain star mona washbourn paul nichola vanessa howard 1 part abandon attempt amicu pictur compet hammer studio break grindhous market studio execut ultim disturb final product releas amicu name sold film 20th centuri fox'

In [37]:
import string
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re

def preprocess(sentence):
    sentence = sentence.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(sentence)
    filtered_words = [w for w in tokens if not w in stopwords.words('english')]
    return " ".join(filtered_words)

sentence = "At eight \n o'clock on Thursday morning Arthur didn't feel very good. French-Fries"
print(preprocess(sentence))


eight clock thursday morning arthur feel good french fries


# NOw I create the dictionary

In [54]:
dictionar = {}
k = 0
for i in range(10000):
    file1 = open("Cleantsv/filmclean-"+str(i)+'.tsv', encoding="utf8") 
    line = file1.read()# Use this to read file content as a stream: 
    #print(line)
    words = line.split('\t') 
    wordssplitted1 = words[14].split()
    wordssplitted2 = words[15].split()
    #print(wordssplitted1, wordssplitted2)
    for i in wordssplitted1:
        #print(type(i))
        if i not in dictionar:
            dictionar[i] = str(k)
            k = k+1
    for i in wordssplitted2:
        #print(type(i))
        if i not in dictionar:
            dictionar[i] = str(k)
            k = k+1
#dictionar
import json

with open('Dictionary.json', 'w') as fp:
    json.dump(dictionar, fp)

In [9]:
import pandas as pd
with open(r"C:\Users\leona\Desktop\ADMHMK-3\Dictionary.json", 'r') as file:
    data = file.read()
diction = json.loads(data)
#dictions = pd.DataFrame(diction)
#diction

In [21]:
len(diction)
type(diction['hom'])

str

# Here is the inverted Dictionary

In [40]:
dictionar2 = {}
length = 0
for i in range(10000):
    file = "Cleantsv/filmclean-"+str(i)+'.tsv'
    file1 = open(file, encoding="utf8") 
    line = file1.read()# Use this to read file content as a stream: 
    #print(line)
    words = line.split('\t') 
    wordssplitted1 = words[14].split()
    wordssplitted2 = words[15].split()
    #print(wordssplitted1, wordssplitted2)
    for j in wordssplitted1:
        code = diction[j]
        if code not in dictionar2:
            dictionar2[code] = [file]
        elif file not in dictionar2[code]:
            dictionar2[code].append(file)
    for j in wordssplitted2:
        code = diction[j]
        if code not in dictionar2:
            dictionar2[code] = [file]
        elif file not in dictionar2[code]:
            dictionar2[code].append(file)
#dictionar
import json

with open('Dictionary2.json', 'w') as fp:
    json.dump(dictionar2, fp)

In [41]:
import pandas as pd
with open(r"C:\Users\leona\Desktop\ADMHMK-3\Dictionary2.json", 'r') as file:
    data = file.read()
diction2 = json.loads(data)
#dictions = pd.DataFrame(diction)
diction2['0']

['Cleantsv/filmclean-0.tsv',
 'Cleantsv/filmclean-22.tsv',
 'Cleantsv/filmclean-24.tsv',
 'Cleantsv/filmclean-28.tsv',
 'Cleantsv/filmclean-41.tsv',
 'Cleantsv/filmclean-48.tsv',
 'Cleantsv/filmclean-50.tsv',
 'Cleantsv/filmclean-59.tsv',
 'Cleantsv/filmclean-67.tsv',
 'Cleantsv/filmclean-80.tsv',
 'Cleantsv/filmclean-81.tsv',
 'Cleantsv/filmclean-85.tsv',
 'Cleantsv/filmclean-87.tsv',
 'Cleantsv/filmclean-88.tsv',
 'Cleantsv/filmclean-90.tsv',
 'Cleantsv/filmclean-95.tsv',
 'Cleantsv/filmclean-106.tsv',
 'Cleantsv/filmclean-107.tsv',
 'Cleantsv/filmclean-119.tsv',
 'Cleantsv/filmclean-128.tsv',
 'Cleantsv/filmclean-147.tsv',
 'Cleantsv/filmclean-156.tsv',
 'Cleantsv/filmclean-162.tsv',
 'Cleantsv/filmclean-168.tsv',
 'Cleantsv/filmclean-169.tsv',
 'Cleantsv/filmclean-170.tsv',
 'Cleantsv/filmclean-183.tsv',
 'Cleantsv/filmclean-185.tsv',
 'Cleantsv/filmclean-194.tsv',
 'Cleantsv/filmclean-204.tsv',
 'Cleantsv/filmclean-227.tsv',
 'Cleantsv/filmclean-235.tsv',
 'Cleantsv/filmclean-244.